In [2]:
"""
This notebook is to look at the data and to try to mimic the controller. The reason for this is that the laser does not
accept the commands for some reason when it is on. Must look at the pattern between the controller and the laser and at what intervals
they send each other messages, and which messages they send to each other.
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [19]:
idle_main_menu = pd.read_csv('idle_main_menu.csv', delimiter = ",")
idle_main_menu

,Timestamp,Arbitration_id,dlc,data
0,1.757712e+09,0x18820810,5,020a016f09
1,1.757712e+09,0x18820810,3,070101
2,1.757712e+09,0x18820810,8,102045204f464620
3,1.757712e+09,0x18820810,4,00202000
4,1.757712e+09,0x18820810,5,0200007a20
...,...,...,...,...
3746,1.757712e+09,0x19000410,3,0a1fff
3747,1.757712e+09,0x19000410,3,021fff
3748,1.757712e+09,0x19000410,3,071fff
3749,1.757712e+09,0x19000410,3,101fff


In [6]:
def device_division(data):
    """
    Enter a dataset and it will return the data divided into the corresponding device.
    "0x18820810" --> The laser
    "0x19000410" --> The controller
    """
    laser = data[data['Arbitration_id'] == '0x18820810']
    controller = data[data['Arbitration_id'] == '0x19000410']
    return laser, controller
    

In [7]:
def different_values(Idle_data, command_data):
    laser_idle = Idle_data[Idle_data['Arbitration_id'] == '0x18820810']
    controller_idle = Idle_data[Idle_data['Arbitration_id'] == '0x19000410']
    
    laser = command_data[command_data['Arbitration_id'] == '0x18820810']
    controller = command_data[command_data['Arbitration_id'] == '0x19000410']

    Lidle_data_unique, Lidle_data_counts = np.unique(laser_idle['data'], return_counts = True)
    Cidle_data_unique, Cidle_data_counts = np.unique(controller_idle['data'], return_counts = True)

    Lcommand_data_unique, Lcommand_data_counts = np.unique(laser['data'], return_counts = True)
    Ccommand_data_unique, Ccommad_data_counts = np.unique(controller['data'], return_counts = True)

    # Now will add the different commands to a list to compare
    similar = []
    different = []

    for message in Ccommand_data_unique:
        if message in Cidle_data_unique:
            similar.append(message)
        else:
            different.append(message)
    # See how many times each one showed
    return similar, different
    

In [10]:
laser_idle, controller_idle = device_division(idle_main_menu)
laser_idle.head(10)

,Timestamp,Arbitration_id,dlc,data
0,1.757712e+09,0x18820810,5,020a016f09
1,1.757712e+09,0x18820810,3,070101
2,1.757712e+09,0x18820810,8,102045204f464620
3,1.757712e+09,0x18820810,4,00202000
4,1.757712e+09,0x18820810,5,0200007a20
5,1.757712e+09,0x18820810,5,02010c3b09
6,1.757712e+09,0x18820810,3,070101
8,1.757712e+09,0x18820810,8,10506d702020302e
9,1.757712e+09,0x18820810,4,00302500
10,1.757712e+09,0x18820810,5,0200007a20


In [11]:
controller_idle.head(10)

,Timestamp,Arbitration_id,dlc,data
7,1.757712e+09,0x19000410,3,021fff
27,1.757712e+09,0x19000410,3,071fff
28,1.757712e+09,0x19000410,3,101fff
29,1.757712e+09,0x19000410,3,101fff
30,1.757712e+09,0x19000410,3,021fff
31,1.757712e+09,0x19000410,3,071fff
32,1.757712e+09,0x19000410,3,021fff
33,1.757712e+09,0x19000410,3,101fff
34,1.757712e+09,0x19000410,3,101fff
35,1.757712e+09,0x19000410,3,021fff


In [16]:
laser_unique_data, laser_counts_data = np.unique(laser_idle['data'], return_counts = True)
laser_unique_data

array(['00202000', '0020312000', '00302500', '00f84300', '0200007a20',
       '02010c3b09', '0201163b09', '020a016f09', '023e0c3b09', '070101',
       '070202', '0a0000000901', '0a00003b0001', '0d00', '0d01', '101e00',
       '102045204f464620', '1020506b20202020', '10506d702020302e',
       '1074202020302e30', '12'], dtype=object)

In [17]:
controller_unique_data, controller_counts_data = np.unique(controller_idle['data'], return_counts = True)
controller_unique_data

array(['021fff', '071fff', '0a1fff', '0d1fff', '101fff', '121fff'],
      dtype=object)

In [21]:
# Now comparing the idle between the idle in main menu and idle in the burst mode menu
idle_burst_menu = pd.read_csv('can_log.csv', delimiter = ",")
idle_burst_menu.columns = ['Timestamp', 'Arbitration_id', 'dlc', 'data']


In [22]:
similar, different = different_values(idle_burst_menu, idle_main_menu)
print(similar)
print(different)
# It seems that they are similar, so it doesn't matter if the controller is on any screen.

['021fff', '071fff', '0a1fff', '0d1fff', '101fff', '121fff']
[]
